In [ ]:
import sys

import requests
import os
import shutil
from time import sleep
from math import ceil
import pickle
import matplotlib.pyplot as plt
import numpy as np

import ee
from osgeo import gdal, ogr, osr
from skimage.restoration import denoise_tv_bregman
from googleapiclient.discovery import build
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.signal import convolve

#run this line if you've never authenticated earthengine
#ee.Authenticate()

ee.Initialize()

# Constants

In [ ]:
# this is about 30 meters
RESOLUTION = 1/3600

#the value we use to signify no data at a pixel
NO_DATA_VALUE = 65535

#bands to despeckle
BANDS_TO_DESPECKLE = ['HH', 'HV']

#store the roi data here
ROI_DATA_FOLDER = 'roi_data'

#store the suggested roi classes here
ROI_SUGGESTED_FOLDER = 'roi_suggested'

#store the predicted roi classes here
ROI_PREDICTED_FOLDER = 'roi_predicted'

for FOLDER in [ROI_DATA_FOLDER, ROI_SUGGESTED_FOLDER, ROI_PREDICTED_FOLDER]:
    #create training data folder if not exists
    if FOLDER not in os.listdir():
        os.mkdir(FOLDER)
    
#features to extract from GEE in the training process
FEATURES = {
            ('collection','JAXA/ALOS/PALSAR/YEARLY/SAR'): ['HH', 'HV', 'qa'], 
            ('collection', 'LANDSAT/LC08/C01/T1_8DAY_NDVI'): ['NDVI'], 
            ('collection', 'LANDSAT/LC08/C01/T1_8DAY_NDWI'): ['NDWI'],
            ('collection', 'LANDSAT/LC08/C01/T1_SR'): ['B3', 'B2', 'B4', 'B5', 'pixel_qa'],
            ('image','CGIAR/SRTM90_V4'): ['elevation']
           }

#the shapefile storing training polygons
TRAINING_POLYGONS_FILE = 'C:/Users/ritvik/Desktop/JPLProject/mapping-colombia-wetlands/training_polygons/training_polygons.shp'

#the shapefile storing roi polygons
ROI_POLYGONS_FILE = 'C:/Users/ritvik/Desktop/JPLProject/mapping-colombia-wetlands/roi_polygons/roi_polygons.shp'

# User Input Area

In [ ]:
#the list of bands to use for training. Choose from:
#Landsat: ['NDVI', 'NDWI', 'B2', 'B3', 'B4', 'B5']
#ALOS-2: ['HH', 'HV']
#CGIAR: ['elevation']

SELECTED_BANDS = ['NDVI', 'NDWI', 'B3', 'B2', 'B4', 'elevation']

#update the features dictionary
FEATURES = {collection: bands for collection,bands in FEATURES.items() if len([b for b in SELECTED_BANDS if b in bands]) != 0 or collection == ('image','CGIAR/SRTM90_V4')}

In [ ]:
#any pixels above this elevation (in meters) will be disregarded from training 
MAX_CONSIDERED_ELEVATION = 50

In [ ]:
#data date range
DATE_RANGE = ['2017-01-01', '2020-01-01']

In [ ]:
#method to use for prediction. Choices are 'histogram' or 'random_forest'
METHOD = 'random_forest'

In [ ]:
#whether or not this notebook will be online. 
#If True then we will download data from Google Earth Engine, requiring an internet connection.
#If False, the user is expected to populate the folder "roi_data" with existing rasters. No internet connection needed.
ONLINE = True

In [ ]:
#the folder id in Google Drive where to temporarily store the GEE data before locally downloading
GOOGLE_EARTH_ENGINE_GDRIVE_FOLDER_ID = '1KvlrUHs_rN7xPlw53qtd9pweeLwmrJSP'

#the name of that same Google Drive folder
GDRIVE_FOLDER_NAME = 'GoogleEarthEngine'

# General Purpose Functions to Read Google Drive Files

In [ ]:
def download_file_from_google_drive(file_id, destination):
    URL = "https://docs.google.com/uc?export=download"
    
    max_tries = 10
    curr_try = 0
    status_code = -1
    
    while status_code != 200 and curr_try < max_tries:
        if curr_try > 0:
            sleep(30)
        session = requests.Session()
        response = session.get(URL, params = { 'id' : file_id }, stream = True)
        status_code = response.status_code
        curr_try += 1
        
    if status_code != 200:
        return
    
    token = get_confirm_token(response)

    if token:
        params = { 'id' : file_id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

In [ ]:
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

In [ ]:
def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
def get_file_ids_from_google_drive():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    service = build('drive', 'v3', credentials=creds)

    result = service.files().list(q="parents in '%s'"%GOOGLE_EARTH_ENGINE_GDRIVE_FOLDER_ID).execute()

    file_name_to_file_id = {info['name']: info['id'] for info in result['files']}
    
    return file_name_to_file_id

In [ ]:
def delete_file_from_google_drive_by_file_id(fid):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    service = build('drive', 'v3', credentials=creds)
    
    service.files().delete(fileId=fid).execute()

# General Purpose Functions to Download Data From Google Earth Engine

In [ ]:
def mask_landsat_sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()

    # Get the pixel QA band.
    qa = image.select('pixel_qa')

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

    return image.updateMask(mask)

In [ ]:
def add_quality_bands(image):
    return image.addBands(image.normalizedDifference(['B5', 'B4']))

In [ ]:
def get_training_data(polygon_features, features, gdrive_folder, date_range, selected_bands):
    """
    This function accepts the below parameters and querys Google Earth Engine for data. The data is stored in 
    Google Drive.
    
    Inputs:
        polygon_features: a list of pairs like (index, polygon feature) indicating which polygons of data to download
        features: the features to extract from GEE
        gdrive_folder: the name of the folder in Google Drive where the downloaded data will live
        date_range: the start and end date to gather data
        selected_bands: list of bands to subset
        
    Output:
        list of tasks which are ready to be started
    """
    
    #this will store all started tasks
    tasks = {}
    
    #work through each sub-region 
    for curr_idx, polygon_feature in polygon_features:
        
        skip_polygon = False
        
        filtered_imgs = []

        #store the reference coordinates
        x1 = polygon_feature.GetGeometryRef().GetEnvelope()[0]
        y1 = polygon_feature.GetGeometryRef().GetEnvelope()[2]
        ref_coords = (x1,y1)
        
        #generate file name
        fname = hash(''.join([str(item) for item in polygon_feature.GetGeometryRef().GetEnvelope()]))
        
        #get polygon area coordinates
        area_coords = [list(pair) for pair in polygon_feature.GetGeometryRef().GetBoundary().GetPoints()]

        #create an area of interest from Earth Engine Geometry
        area_of_interest = ee.Geometry.Polygon(coords=area_coords)

        #iterate over each data source
        for data_type_source, bands in features.items():
            data_type = data_type_source[0]
            data_source = data_type_source[1]
                
            print('Working on data source: %s...'%data_source)
            
            if data_type == 'collection':
                #access the Earth Engine image collection with the specified bands
                data = ee.ImageCollection(data_source).select(bands)

                #filter on date range and area of interest
                data_filtered = data.filterBounds(area_of_interest).filterDate(date_range[0], date_range[1])
                
                #if data source is landsat SR, further filter by cloud cover
                if data_source == 'LANDSAT/LC08/C01/T1_SR':
                    data_filtered = data_filtered.filterMetadata('CLOUD_COVER', 'less_than', 50).map(mask_landsat_sr).map(add_quality_bands)
                    
                #ensure there is at least 1 image
                num_items = data_filtered.size().getInfo()
                print('%s images in collection'%num_items)
                if num_items == 0:
                    skip_polygon = True
                    break

                #if LANDSAT, quality mosaic by its respective metric
                if 'LANDSAT' in data_source:
                    #derived LANDSAT band
                    if '8DAY' in data_source:
                        mosaic = data_filtered.qualityMosaic(features[data_type_source][0])
                    #LANDSAT Surface Reflectance
                    else:
                        mosaic = data_filtered.qualityMosaic('nd')
                #otherwise just do a simple median
                else:
                    mosaic = data_filtered.median()
                   
            elif data_type == 'image':
                mosaic = ee.Image(data_source).select(bands)

            #add this mosaic to the list
            filtered_imgs.append(mosaic)
        
        if skip_polygon:
            print('Skipping Polygon%s'%curr_idx)
            tasks[fname] = None
            print('==================================')
            continue
        
        #add the various layers on top of each other to create a data cube with all features
        final_img = ee.Image()
        
        for img in filtered_imgs:
            final_img = ee.Image.addBands(final_img,img)
        
        #use the ALOS qa band to filter out invalid pixels
        if ('collection','JAXA/ALOS/PALSAR/YEARLY/SAR') in features and 'qa' in features[('collection','JAXA/ALOS/PALSAR/YEARLY/SAR')]:
            qa_band = final_img.select('qa')
            qa_mask = qa_band.eq(0)
            final_img = final_img.where(qa_mask, NO_DATA_VALUE)
            
        #use the SRTM elevation band to filter out invaild pixels
        if ('image','CGIAR/SRTM90_V4') in features and 'elevation' in features[('image','CGIAR/SRTM90_V4')]:
            elevation_band = final_img.select('elevation')
            elevation_mask = elevation_band.gt(MAX_CONSIDERED_ELEVATION)
            final_img = final_img.where(elevation_mask, NO_DATA_VALUE)
            
        #if any of the selected bands has NO_DATA_VALUE, mark that whole pixel as NO_DATA_VALUE
        for b in selected_bands:
            b_values = final_img.select(b)
            b_mask = b_values.eq(NO_DATA_VALUE)
            final_img = final_img.where(b_mask, NO_DATA_VALUE)
         
        #store the result with just the needed bands
        selected_bands = sorted(selected_bands)
        result = final_img.select(*selected_bands).float()
          
        #define the task to gather the data
        task = ee.batch.Export.image.toDrive(image=result,
                                             region=area_of_interest.getInfo()['coordinates'],
                                             description=str(curr_idx),
                                             folder=gdrive_folder,
                                             fileNamePrefix=str(fname),
                                             crs_transform=[RESOLUTION, 0.0, ref_coords[0], 0.0, -RESOLUTION, ref_coords[1]],
                                             crs='EPSG:4326')
        
        #store the task
        tasks[fname] = task
        
        print('==================================')
        
    return list(tasks.items())

In [ ]:
def execute_tasks_in_batches(tasks, batch_size, FOLDER, wait_seconds=30):
    """
    Executes a list of tasks in batches
    
    Inputs:
        tasks: list of tasks
        batch_size: number of tasks to execute per batch
        FOLDER: the folder to store the downloaded rasters
    """
    
    fnames = []

    #process the tasks in small batches to avoid memory running out
    for batch_idx in range(ceil(len(tasks) / batch_size)):

        #get the current batch of tasks
        curr_tasks = tasks[batch_size*batch_idx:batch_size*(batch_idx+1)]
        print('Processing Batch %s'%(batch_idx+1))

        #start all tasks in that batch
        for name,task in curr_tasks:
            if task != None:
                task.start()

        print('Started all tasks in batch')

        #wait until all tasks in that batch are done
        curr_states = [task.status()['state'] for name,task in curr_tasks if task != None]
        while 'RUNNING' in set(curr_states) or 'READY' in set(curr_states):
            print('Current states: %s'%curr_states)
            sleep(wait_seconds)
            curr_states = [task.status()['state'] for name,task in curr_tasks if task != None]

        #once all tasks done, get their file ids on google drive
        file_name_to_file_id = get_file_ids_from_google_drive()

        #for each file...
        for fname, fid in file_name_to_file_id.items():

            #get feature file name
            features_file_name = '%s/features_%s'%(FOLDER, fname)

            #check if data already downloaded
            print('Downloading %s from Drive'%fname)
            download_file_from_google_drive(fid, features_file_name)

            print('Deleting %s from Drive'%fname)
            delete_file_from_google_drive_by_file_id(fid)
            
            fnames.append(features_file_name)

        print('================================')
        
    return fnames

# General Purpose Despeckling, Clustering, and Raster Functions

In [ ]:
def img_to_db(img):
    return 10 * np.log10(img+.00001)

def db_to_img(img):
    return 10**(img / 10)

def tv_denoise(arr, idxs_to_despeckle, weight):
    copy_arr = arr.copy()
    for idx in idxs_to_despeckle:
        #get the layer
        layer = copy_arr[:,:,idx]
        
        orig_valid_mask = ~np.isnan(layer)
        
        #denoise
        img_db = img_to_db(layer)
        img_db_tv = denoise_tv_bregman(img_db, weight)
        img_tv = db_to_img(img_db_tv)
        img_tv[orig_valid_mask & np.isnan(img_tv)] = layer[orig_valid_mask & np.isnan(img_tv)]
        
        #set denoised into copy of array
        copy_arr[:,:,idx] = img_tv
        
    return copy_arr

In [ ]:
def get_sep_metric(feat_file_name, num_clusters_options):
    """
    Get the separability metric for the given array and given possible clusters
    
    Inputs:
        feat_file_name: path to file to analyze
        num_clusters_options: list of number of clusters to try
        
    """
    
    #read the array
    ds = gdal.Open('%s/%s'%(ROI_DATA_FOLDER, feat_file_name), gdal.GA_ReadOnly)

    #get gt and read array
    gt = ds.GetGeoTransform()
    arr = ds.ReadAsArray()
    arr = np.stack([arr[i] for i in range(arr.shape[0])], axis=-1)
    ds = None

    #transform array to 2d
    arr_2d = arr.reshape(-1,arr.shape[-1])
    num_bands = arr_2d.shape[-1]
    arr_2d[arr_2d == NO_DATA_VALUE] = np.nan
    valid_indices = np.where(np.all(~np.isnan(arr_2d), axis=-1))[0]
    arr_2d_valid = arr_2d[valid_indices]
    
    #this will store the separability metric values for differet k
    sep_metric_dict = {}
    
    #for each value of k (number clusters)
    for k in num_clusters_options:
        
        print('trying %s clusters...'%k)

        #define model
        model = KMeans(n_clusters=k)

        #fit model and predict clusters
        cluster_preds = model.fit_predict(arr_2d_valid)

        #these will store the average and deviation of each band for each cluster
        mu_vals = np.zeros((k,num_bands))
        dev_vals = np.zeros((k,num_bands))

        #populate above arrays
        for cid in range(k):
            cluster_data = arr_2d_valid[cluster_preds == cid]
            mu, dev = cluster_data.mean(axis=0), cluster_data.std(axis=0)
            mu_vals[cid] = mu
            dev_vals[cid] = dev

        #get pairwise separability metric values and put in list
        sep_metric_vals = []
        for c1 in range(k):
            for c2 in range(c1+1,k):
                sep_metric_vals.append(np.mean(abs(mu_vals[c1] - mu_vals[c2]) / (dev_vals[c1] + dev_vals[c2])))

        #add information to dictionary
        sep_metric_dict[k] = {'sep_metric_vals': sep_metric_vals, 'cluster_preds': cluster_preds}
    
    #best separating k has the highest median separation between each pair of clusters
    best_sep_entry = sorted(sep_metric_dict.items(), key=lambda info: -np.median(info[1]['sep_metric_vals']))[0]
    print('Best Separating k: %s'%str(best_sep_entry[0]))
    
    #put the result in an array for viewing in a GIS editor
    result = np.ones(arr_2d.shape[0])
    result[:] = np.nan
    result[valid_indices] = best_sep_entry[1]['cluster_preds']
    result = result.reshape(arr.shape[:2])
    
    #apply a majority filter to smooth the clusters out
    result = apply_majority_filter(result, 5)

    #save to array
    ds = np_array_to_raster('%s/suggested_%s'%(ROI_SUGGESTED_FOLDER, feat_file_name), result, ['class_id'], gt, no_data=-1, nband=1, gdal_data_type=gdal.GDT_Float64)
    ds = None

In [ ]:
def get_bounding_feature_around_raster(raster_fname):
    """
    Gets a bounding osgeo.ogr.Feature around raster with given file name
    """
    
    #get raster
    raster = gdal.Open(raster_fname, gdal.GA_ReadOnly)

    # Get raster geometry
    transform = raster.GetGeoTransform()

    pixelWidth = transform[1]
    pixelHeight = transform[5]
    cols = raster.RasterXSize
    rows = raster.RasterYSize

    raster = None

    #get extents
    xLeft = transform[0]
    yTop = transform[3]
    xRight = xLeft+cols*pixelWidth
    yBottom = yTop+rows*pixelHeight

    #form raster geometry
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(xLeft, yTop)
    ring.AddPoint(xLeft, yBottom)
    ring.AddPoint(xRight, yBottom)
    ring.AddPoint(xRight, yTop)
    ring.AddPoint(xLeft, yTop)

    rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
    rasterGeometry.AddGeometry(ring)
    
    return rasterGeometry

In [ ]:
def remove_temporary_files():
    """
    removes temporary files created in the prediction process. 
    These files are designated to start with "temp_"
    """
    
    #remove temporary files
    for temp_fname in [fname for fname in os.listdir() if 'temp_' in fname]:
        try:
            os.remove(temp_fname)
        except FileNotFoundError:
            pass
        except PermissionError:
            print('Failed to remove %s due to PermissionError; please attempt to remove manually'%temp_fname)

In [ ]:
def apply_majority_filter(arr, ksize):
    """
    Apply majority filter to array of predicted class ids. Predicted labels assumed to be >= 0.
    
    Inputs:
        arr: the array
        ksize: the size of the kernel. Bigger means more neighboring pixels considered in the decision
    
    """
    
    #this kernel will be used for the majority filter on predicted class id
    ksize = 5
    kernel = np.ones((ksize,ksize)) / ksize**2
    
    #get the original NaN values
    mask = np.isnan(arr)
    
    #get unique class ids
    class_ids = np.sort(np.unique(arr[arr >= 0]).astype(int))
    class_scores = np.zeros(arr.shape + (len(class_ids),))
    
    #get the matrix of T/F wheter each pixel is predicted as each class
    class_mtxs = {cid: (arr == cid) for cid in class_ids}
    
    #for each class id...
    for cid in class_ids:
        #convolve with the low pass filter
        conv_mtx = convolve(class_mtxs[cid], kernel, mode='same')
        conv_mtx[np.isnan(arr)] = np.nan
        class_scores[:,:,cid] = conv_mtx
    
    filtered_arr = np.argmax(class_scores, axis=-1).astype(float)
    filtered_arr[mask] = np.nan
    
    return filtered_arr

# Functions to Process Feature Files

In [ ]:
def process_feature_files(feat_file_names):
    """
    This function accepts a list of file names and processes those rasters. 
    
    Inputs:
        feat_file_names: a list of names of the downloaded training data files
        confidence_levels: a list of confidence levels associated with each file in feat_file_names
        preprocess: whether to remove some pixels that likely do not belong
        
    Outputs:
        the processed training data and auxilary data such as geotransforms
    """
    
    #this will store the numpy array of training data for each file
    data = {}
    
    #this will store auxilary data for each file
    feat_file_data = {}
    
    #iterate over each file
    for feat_file_name in feat_file_names:
        
        feat_file_name = '%s/%s'%(ROI_DATA_FOLDER, feat_file_name)

        #open file and get geotransform
        orig_ds = gdal.Open(feat_file_name, gdal.GA_ReadOnly)
        
        try:
            gt = orig_ds.GetGeoTransform()
        except AttributeError:
            print('Could not process %s'%feat_file_name)
            continue

        #despeckle any bands which need to be despeckled
        band_names = [orig_ds.GetRasterBand(idx+1).GetDescription() for idx in range(orig_ds.RasterCount)]
        idx_to_despeckle = [idx for idx in range(orig_ds.RasterCount) if orig_ds.GetRasterBand(idx+1).GetDescription() in BANDS_TO_DESPECKLE]
        arr = orig_ds.ReadAsArray()
        arr = np.stack([arr[i] for i in range(arr.shape[0])], axis=-1)
        data_mask = np.any(arr == NO_DATA_VALUE, axis=-1) | np.any(np.isnan(arr), axis=-1)
        arr[data_mask] = NO_DATA_VALUE

        arr = tv_denoise(arr, idx_to_despeckle, 1)
        arr[data_mask] = np.nan

        orig_ds = None
        
        out_ds = np_array_to_raster(feat_file_name, arr, band_names, gt, no_data=NO_DATA_VALUE, nband=arr.shape[-1], gdal_data_type=gdal.GDT_Float64)
        out_ds = None
        
        print('Processed %s'%feat_file_name)

In [ ]:
def create_histograms(training_datasets):
    """
    Creates a set of histograms, one for each class
    
    Inputs:
        training_histograms: a dictionary mapping class_id to a dataset
        
    Outputs:
        a dictionary mapping class_id to a histogram
        a list of histogram bin cuttoffs for each feature
    """
    
    num_features = list(training_datasets.values())[0].shape[-1]
    
    min_feature_values = [min([min(dataset[:,idx]) for dataset in training_datasets.values()]) for idx in range(num_features)]
    max_feature_values = [max([max(dataset[:,idx]) for dataset in training_datasets.values()]) for idx in range(num_features)]
    
    histogram_ranges = []
    training_histograms = {}
    num_bins = 5

    for idx in range(num_features):
        width = (max_feature_values[idx] - min_feature_values[idx]) / num_bins
        histogram_ranges.append(np.arange(min_feature_values[idx], max_feature_values[idx]+width*.99, width))
    histogram_ranges = np.array(histogram_ranges)

    for class_id in class_ids:
        training_histograms[class_id] = np.histogramdd(training_datasets[class_id], bins=histogram_ranges, density=True)[0] 
        
    return training_histograms, histogram_ranges

In [ ]:
def get_classes(test_set, training_histograms, histogram_ranges, class_ids, frac=0.25):
    """
    This function accepts a data set and histograms and classifies each pixel and assigns a score
    
    Inputs:
        test_set: the test set of features which we would like to classify
        training_histograms: a dictionary of histograms, one for each class
        histogram_ranges: the bin cuttoffs for the histograms
        class_ids: a set of class ids
        frac: between 0 and 1. Higher values mean we require more confidence to classify a pixel as non-NaN
        
    Output:
        an array of predicted classes and corresponding scores
    """
    
    #this will store the probabilities for each class
    class_id_to_probs = {}
    
    #any probability density below this is considered as 0
    min_allowable_density = min([frac*np.max(training_histograms[cid]) for cid in class_ids])
    
    #iterate over each histogram
    for class_id, histogram in training_histograms.items():
        
        #get the position of each test pixel in the context of this histogram
        transposed_ranges = np.transpose(histogram_ranges)
        expanded_dims_ranges = np.expand_dims(transposed_ranges, axis=1)
        extended_ranges = np.concatenate([expanded_dims_ranges for _ in range(test_set.shape[0])], axis=1)
        diffs = extended_ranges - test_set
    
        #get the probability density at each position
        indices = np.argmax(diffs > 0, axis=0) - 1
        indices[indices < 0] = 0
        indices = indices - np.all((diffs>0)==False, axis=0)
        indices = tuple(np.transpose(indices))
        probs = histogram[indices]
        
        #store this in the dictionary
        class_id_to_probs[class_id] = probs
      
    #create matrix of probabilities for each class
    prob_mtx = np.stack([class_id_to_probs[cid] for cid in class_ids], axis=-1)
    
    #sort matrix of probs
    sorted_probs = np.sort(prob_mtx, axis=1)
    
    #any pixel where all classes have 0 probability is NaN
    nan_indices = np.where(sorted_probs[:,-1] < min_allowable_density)[0]
    
    #compute scores based on ratio of most likely class to second most likely class
    scores = 1/(1+np.exp(-(sorted_probs[:,-1] / sorted_probs[:,-2])))
    
    #apply NaN pixels
    scores[nan_indices] = np.nan

    #get the predicted class
    pred_class = np.argmax(prob_mtx, axis=1).astype(float)
    
    #apply NaN pixels
    pred_class[nan_indices] = np.nan
    
    return pred_class, scores

# Numpy Array to TIFF Functions

In [ ]:
def create_raster(output_path, columns, rows, nband=1, gdal_data_type=gdal.GDT_Int32, driver=r'GTiff'):
    ''' 
    returns gdal data source raster object 
    '''
    
    # create driver
    driver = gdal.GetDriverByName(driver)

    output_raster = driver.Create(output_path, columns, rows, nband, eType = gdal_data_type)    
    
    return output_raster

def np_array_to_raster(output_path, arr, band_names, geotransform, no_data=None, nband=1, gdal_data_type=gdal.GDT_Int32, spatial_reference_system_wkid=4326, driver=r'GTiff'):
    ''' 
    returns a gdal raster data source

    keyword arguments:

    output_path -- full path to the raster to be written to disk
    numpy_array -- numpy array containing data to write to raster
    band_names -- the names of the bands. must be same length as last axis of numpy_array
    upper_left_tuple -- the upper left point of the numpy array (should be a tuple structured as (x, y))
    cell_resolution -- the cell resolution of the output raster
    no_data -- value in numpy array that should be treated as no data
    nband -- the band to write to in the output raster
    gdal_data_type -- gdal data type of raster (see gdal documentation for list of values)
    spatial_reference_system_wkid -- well known id (wkid) of the spatial reference of the data
    driver -- string value of the gdal driver to use
    '''

    rows, columns = arr.shape[0], arr.shape[1]

    # create output raster
    output_raster = create_raster(output_path, columns, rows, nband, gdal_data_type) 

    spatial_reference = osr.SpatialReference()
    spatial_reference.ImportFromEPSG(spatial_reference_system_wkid)
    output_raster.SetProjection(spatial_reference.ExportToWkt())
    output_raster.SetGeoTransform(geotransform)
    
    for band_idx in range(1,nband+1):
        output_band = output_raster.GetRasterBand(band_idx)
        output_band.SetDescription(band_names[band_idx-1])
        if no_data != None:
            output_band.SetNoDataValue(no_data)
        if nband > 1:
            output_band.WriteArray(arr[:,:,band_idx-1])
        else:
            output_band.WriteArray(arr)
        output_band.FlushCache() 
        output_band.ComputeStatistics(False)

    if os.path.exists(output_path) == False:
        raise Exception('Failed to create raster: %s' % output_path)

    return output_raster

# Driver Code : Download ROI Polygon Data

## This code will download data from Google Earth Engine according to the ROI polygons in the folder "roi_polygons"

In [ ]:
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(ROI_POLYGONS_FILE, gdal.GA_ReadOnly)
layer = dataSource.GetLayer()

#get all roi polygons
roi_polygon_features = [layer.GetNextFeature() for _ in range(layer.GetFeatureCount())]

In [ ]:
#this dictionary maps roi polygons to the corresponding raster file name
roi_polygon_to_fname = {}

existing_raster_names = os.listdir(ROI_DATA_FOLDER)

#for all existing rasters...
for fname in existing_raster_names:
    fname = '%s/%s'%(ROI_DATA_FOLDER, fname)
    
    #get the raster geometry
    raster_geometry = get_bounding_feature_around_raster(fname)
    feature_defn = ogr.FeatureDefn()
    raster_feat = ogr.Feature(feature_defn)
    raster_feat.SetGeometry(raster_geometry)
    
    #if we chose ONLINE mode, then try to match roi polygons to exsting rasters
    if ONLINE:
        
        #for each roi polygon...
        for roi_polygon in roi_polygon_features:

            #if this roi_polygon bounds the current raster
            if roi_polygon.GetGeometryRef().Centroid().Within(raster_feat.GetGeometryRef()):
                roi_polygon_to_fname[roi_polygon] = fname
            
    
    #we failed to find any roi polygon which contains this raster
    if fname not in roi_polygon_to_fname.values():
        #generate a new roi polygon
        roi_polygon_to_fname[raster_feat] = fname

In [ ]:
new_fnames = []
if ONLINE:
    roi_polygon_features_to_process = [p for p in roi_polygon_features if p not in roi_polygon_to_fname]
    tasks = get_training_data(enumerate(roi_polygon_features_to_process), FEATURES, GDRIVE_FOLDER_NAME, DATE_RANGE, SELECTED_BANDS)
    new_fnames = execute_tasks_in_batches(tasks, 3, ROI_DATA_FOLDER, 10)
    for poly,fname in zip(roi_polygon_features_to_process, new_fnames):
        roi_polygon_to_fname[poly] = fname

In [ ]:
#get all files in the roi data folder
roi_feat_file_names = [item for item in os.listdir(ROI_DATA_FOLDER) if item[-4:] == 'tiff' or item[-3:] == 'tif']

In [ ]:
#process the newly gathered files
process_feature_files([fname.split('/')[-1] for fname in new_fnames])

# Driver Code : Get Suggested Number of Classes

## This code will run K-Means on each ROI with different values for K. It will report the "best" number of clusters based on a separability metric. The results are stored graphically in the folder "roi_suggested" and can be viewed in a GIS editor like QGIS. 

In [ ]:
num_clusters_options = [6,7,8,9,10]

for feat_file_name in roi_feat_file_names:
    print('Processing %s'%feat_file_name)
    get_sep_metric(feat_file_name, num_clusters_options)
    print('----------------------')

# ------------------------------------------------------------------------------------------------------------

## At this point please pause and review the suggested number of clusters and create your training polygons on the map using a GIS editor like QGIS. A sample file of training polygons is stored in the folder "training_polygons" but you should create your own set of training polygons based on the ROI you chose in the beginning of this notebook. Make sure that each training polygon lives inside a ROI polygon and also ensure that each training polygon you draw has a:

## *class_id* : integer in 0,1,2... indicating which class this training polygon belongs to

## *confidence*: value between 0 and 1 indicating how confident you are about this training polygon belonging to the labeled class_id

# ------------------------------------------------------------------------------------------------------------

# Driver Code : Extract Training Data

## This code will extact data for the training polygons from the existing data for the ROI polygons. It uses this extracted data to create a dictionary mapping ROI polygon to class_id to training data.

In [ ]:
driver = ogr.GetDriverByName('ESRI Shapefile')
dataSource = driver.Open(TRAINING_POLYGONS_FILE, gdal.GA_ReadOnly)
layer = dataSource.GetLayer()
training_polygon_features = [layer.GetNextFeature() for _ in range(layer.GetFeatureCount())]

In [ ]:
#this dictionary maps roi data file name to class_id to training data file name
roi_to_class_id_to_training = {}

#this dictionary maps roi data file name to data information
roi_to_data = {}

remove_temporary_files()

#for each roi polygon...
for roi_polygon, roi_fname in roi_polygon_to_fname.items():
    print('Processing %s...'%roi_fname)
    
    roi_to_class_id_to_training[roi_fname] = {}
    roi_to_data[roi_fname] = {}
    
    #get the dataset
    ds_roi = gdal.Open(roi_fname, gdal.GA_ReadOnly)
    roi_data = ds_roi.ReadAsArray()
    roi_data = np.stack([roi_data[i] for i in range(roi_data.shape[0])], axis=-1)
    
    #mask out NaN or NO DATA values
    mask = np.any(roi_data == NO_DATA_VALUE, axis=-1) | np.any(np.isnan(roi_data), axis=-1)
    roi_data = roi_data[~mask]
    
    #map roi_fname to data and auxilary data information like geotransform, shape, etc.
    roi_to_data[roi_fname]['data'] = roi_data
    roi_to_data[roi_fname]['indices'] = np.where(~mask)
    roi_to_data[roi_fname]['gt'] = ds_roi.GetGeoTransform()
    roi_to_data[roi_fname]['shape'] = mask.shape
    
    #get list of training polygons inside this roi polygon
    contained_training_polygons = [f for f in training_polygon_features if f.GetGeometryRef().Centroid().Within(roi_polygon.GetGeometryRef())]
    
    #get unique class ids within this roi polygon
    class_ids = sorted(list(set([f.GetField('class_id') for f in contained_training_polygons])))
    
    #for each class id...
    for cid in class_ids:
        print('Processing class_id=%s'%cid)
        
        #get list of training polygons of this class
        class_specific_polygons = [f for f in contained_training_polygons if f.GetField('class_id') == cid]
        total_confidence = sum([f.GetField('confidence') for f in class_specific_polygons])
        training_polygon_weights = [f.GetField('confidence') / total_confidence for f in class_specific_polygons]
        
        #initialize the training data for this specific roi polygon and class
        class_specific_training_data = []
        
        #for each individual training polygon...
        for fid, feature in enumerate(class_specific_polygons):
        
            #create a temporary shapefile with just this training polygon
            temp_poly_name = 'temp_poly_%s_%s_%s.shp'%(''.join([c for c in roi_fname if c.isdigit() or c == '-']), cid, fid)
            created_shp_file = driver.CreateDataSource(temp_poly_name)
            outLayer = created_shp_file.CreateLayer('tmp')
            outLayer.CreateFeature(feature)
            created_shp_file.Destroy()

            #get the training data in these polygons
            temp_raster_name = 'temp_raster_%s_%s_%s.tiff'%(''.join([c for c in roi_fname if c.isdigit() or c == '-']), cid, fid)
            options = gdal.WarpOptions(format='GTiff', cutlineDSName=temp_poly_name, srcNodata=NO_DATA_VALUE)
            ds_result = gdal.Warp(destNameOrDestDS=temp_raster_name, srcDSOrSrcDSTab=ds_roi, options=options)
        
            #extract training data as array
            training_data = ds_result.ReadAsArray()
            training_data = np.stack([training_data[i] for i in range(training_data.shape[0])], axis=-1)
            mask = np.any(training_data == NO_DATA_VALUE, axis=-1) | np.any(np.isnan(training_data), axis=-1)
            training_data = training_data[~mask]
            ds_result = None
        
            #add this training data to the list
            class_specific_training_data.append(training_data)
            
            print('Processed feature %s: %s samples'%(fid, training_data.shape[0]))
            
        
        #get the total number of pixels in all training sets
        tot_pixels = sum([ds.shape[0] for ds in class_specific_training_data])
        
        #get number of pixels to sample by considering confidence of each polygon
        pixels_to_sample = [int(tot_pixels*w) for w in training_polygon_weights]
        
        #sample training data by confidence
        class_specific_training_data = [ds[np.random.choice(ds.shape[0], p)] for ds,p in zip(class_specific_training_data, pixels_to_sample) if ds.shape[0] > 0]
        class_specific_training_data = np.concatenate(class_specific_training_data, axis=0)
        
        #run a K-means with 2 clusters in case user accidentially included some noise 
        try:
            model = KMeans(n_clusters=2)
            cluster_preds = model.fit_predict(class_specific_training_data)
        
            #only filter out data if there is a distinctly small second cluster
            if abs(np.mean(cluster_preds) - 0.5) > 0.25:
                main_cluster = np.median(cluster_preds)
                class_specific_training_data = class_specific_training_data[cluster_preds == main_cluster]
        except ValueError:
            pass
        
        #store combined training data for this roi and class_id
        if class_specific_training_data.shape[0] == 0:
            del roi_to_class_id_to_training[roi_fname]
            print('No data gathered for %s'%roi_fname)
            break
            
        roi_to_class_id_to_training[roi_fname][cid] = class_specific_training_data
        print('constructed dataset of size %s'%str(class_specific_training_data.shape))
        
    ds_roi = None
    print('--------------------')
    
remove_temporary_files()

# Driver Code : Classify ROI Areas

## this code builds the model (histogram or random_forest) to predict the ROI areas and then performs the prediction, storing the results in the folder "roi_predicted"

In [ ]:
#create empty histograms and histogram ranges dictionaries
histograms_dict = {}
histogram_ranges_dict = {}

random_forest_clf_dict = {}

for roi_fname in roi_to_class_id_to_training:
    
    class_ids = sorted([cid for cid in roi_to_class_id_to_training[roi_fname].keys() if type(cid) == int])
    
    #create histograms and histogram ranges for this roi polygon
    histograms, histogram_ranges = create_histograms(roi_to_class_id_to_training[roi_fname])
    histograms_dict[roi_fname] = histograms
    histogram_ranges_dict[roi_fname] = histogram_ranges
    
    #store constructed datasets
    features = np.concatenate([roi_to_class_id_to_training[roi_fname][cid] for cid in class_ids], axis=0)
    labels = np.array([single for item in [[cid]*len(roi_to_class_id_to_training[roi_fname][idx]) for idx,cid in enumerate(class_ids)] for single in item])
    
    #fit a random forest classifier
    clf = RandomForestClassifier()
    clf.fit(features, labels)
    random_forest_clf_dict[roi_fname] = clf
    
    print('Processed %s'%roi_fname)

In [ ]:
if METHOD == 'histogram':
    #get the predicted classes and scores for histogram method
    pred_class_scores = {fname: get_classes(d['data'], histograms_dict[fname], histogram_ranges_dict[fname], sorted(list(histograms_dict[fname].keys())), 0) for fname,d in roi_to_data.items()}
elif METHOD == 'random_forest':
    #get the predicted classes and scores for random forest method
    pred_class_scores = {fname: [random_forest_clf_dict[fname].predict(d['data']), np.sort(random_forest_clf_dict[fname].predict_proba(d['data']), axis=1)[:,-2:]] for fname,d in roi_to_data.items()}
    scores = {fname: 2/(1+np.exp(1-item[1][:,-1] / item[1][:,-2]))-1 for fname, item in pred_class_scores.items()}
    pred_class_scores = {fname: [item[0], scores[fname]] for fname, item in pred_class_scores.items()}
    
print('Got Predicted Classes and Scores')

In [ ]:
#for each roi file...
for idx, feat_file_name in enumerate(roi_feat_file_names):
    
    feat_file_name = '%s/%s'%(ROI_DATA_FOLDER, feat_file_name)
    
    #get the shape of the file
    feat_file_shape = roi_to_data[feat_file_name]['shape']
    shape = feat_file_shape + (2,)
    
    #create empty matrix to store result
    result = np.empty(shape)
    result[:] = np.nan
    
    #these are the valid indices 
    indices = roi_to_data[feat_file_name]['indices']

    #load the predicted classes and scores
    result[indices] = np.transpose(pred_class_scores[feat_file_name])
    
    result[:,:,0] = apply_majority_filter(result[:,:,0], 5)

    preds_fname = '%s/%s_predicted_%s.tiff'%(ROI_PREDICTED_FOLDER, METHOD, idx)
    ds = np_array_to_raster(preds_fname, result, ['class_id', 'confidence'], roi_to_data[feat_file_name]['gt'], no_data=-1, nband=2, gdal_data_type=gdal.GDT_Float64)
    ds = None
    print('saved %s'%preds_fname)